In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, torch

In [22]:
df = pd.read_csv('chess_king_rook_dataset.csv')
df.columns

Index(['white_king_file', 'white_king_rank', 'white_rook_file',
       'white_rook_rank', 'black_king_file', 'black_king_rank', 'result'],
      dtype='object')

In [23]:
df.result.value_counts()

fourteen    4553
thirteen    4194
twelve      3597
eleven      2854
draw        2796
fifteen     2166
ten         1985
nine        1712
eight       1433
seven        683
six          592
five         471
sixteen      390
two          246
four         198
three         81
one           78
zero          27
Name: result, dtype: int64

In [24]:
from sklearn.preprocessing import LabelEncoder

def encode_df(dataframe):
    le = LabelEncoder()
    for column in dataframe.columns:
        if 'file' in column:
            dataframe[column] = le.fit_transform(dataframe[column])
            dataframe[column] += 1
    return dataframe

#encode the dataframe
df = encode_df(df)

In [26]:
def encode_results(string):
    if string == 'draw':
        string = -1
    elif string == 'zero':
        string = 0
    elif string == 'one':
        string = 1
    elif string == 'two':
        string = 2
    elif string == 'three':
        string = 3
    elif string == 'four':
        string = 4
    elif string == 'five':
        string = 5
    elif string == 'six':
        string = 6
    elif string == 'seven':
        string = 7
    elif string == 'eight':
        string = 8
    elif string == 'nine':
        string = 9
    elif string == 'ten':
        string = 10
    elif string == "eleven":
        string = 11
    elif string == "twelve":
        string = 12
    elif string == "thirteen":
        string = 13
    elif string == "fourteen":
        string = 14
    elif string == "fifteen":
        string = 15
    else:
        string = 16
    return string
df.result = df.result.apply(lambda x: encode_results(x))
df

,white_king_file,white_king_rank,white_rook_file,white_rook_rank,black_king_file,black_king_rank,result
0,1,1,2,3,3,2,-1
1,1,1,3,1,3,2,-1
2,1,1,3,1,4,1,-1
3,1,1,3,1,4,2,-1
4,1,1,3,2,3,1,-1
...,...,...,...,...,...,...,...
28051,2,1,7,7,5,5,16
28052,2,1,7,7,5,6,16
28053,2,1,7,7,5,7,16
28054,2,1,7,7,6,5,16


In [27]:
df.result.value_counts()

 14    4553
 13    4194
 12    3597
 11    2854
-1     2796
 15    2166
 10    1985
 9     1712
 8     1433
 7      683
 6      592
 5      471
 16     390
 2      246
 4      198
 3       81
 1       78
 0       27
Name: result, dtype: int64

In [30]:
df.isna().sum()

white_king_file    0
white_king_rank    0
white_rook_file    0
white_rook_rank    0
black_king_file    0
black_king_rank    0
result             0
dtype: int64

**Brak NA - teraz policzmy odległość czarnego króla od krawędzi planszy**

In [38]:
def distance(df):
    return np.min([df.black_king_file - 1, 8 - df.black_king_file, df.black_king_rank - 1, 8 - df.black_king_rank])

In [39]:
df['black_king_dst'] = df.apply(lambda x: distance(x))

AttributeError: 'Series' object has no attribute 'black_king_file'

In [35]:
df

,white_king_file,white_king_rank,white_rook_file,white_rook_rank,black_king_file,black_king_rank,result,black_king_dst
0,1,1,2,3,3,2,-1,0
1,1,1,3,1,3,2,-1,0
2,1,1,3,1,4,1,-1,0
3,1,1,3,1,4,2,-1,0
4,1,1,3,2,3,1,-1,0
...,...,...,...,...,...,...,...,...
28051,2,1,7,7,5,5,16,0
28052,2,1,7,7,5,6,16,0
28053,2,1,7,7,5,7,16,0
28054,2,1,7,7,6,5,16,0
